In [1]:
import matplotlib.pyplot as plt
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
import tensorflow as tf
import numpy as np
import glob
import os
import sys  
import cv2
from tqdm import tqdm
import random
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn import preprocessing
import tensorflow as tf
import scipy

from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder





In [2]:
print(tf.__version__)
data_path="npz_data/violence/*"
# data_path2="npz_data/testtalk/*"

img_size=(224,224,3)
frame_number=20
num_class=2



1.13.1


In [3]:

# HEIGHT = 28
# WIDTH = 28
# DEPTH = 3

# NUM_PER_EPOCH = 50000

# def parser(serialized_example):
#     """Parses a single tf.Example into image and label tensors."""
#     features = tf.parse_single_example(
#         serialized_example,
#         features={
#             'image': tf.FixedLenFeature([], tf.string),
#             'label': tf.FixedLenFeature([], tf.string),
#         })
#     image = tf.decode_raw(features['image'], tf.uint8)
#     image.set_shape([924*20*DEPTH * HEIGHT * WIDTH])

#     # Reshape from [depth * height * width] to [depth, height, width].
#     image = tf.cast(
#         tf.transpose(tf.reshape(image, [-1,20, HEIGHT, WIDTH,DEPTH])),
#         tf.float32)
#     label = tf.cast(features['label'], tf.float32)

#     return image, label



# def make_batch(batch_size):
#     """Read the images and labels from 'filenames'."""
#     filenames = [os.path.join("./npz_data/violence", 'train.tfrecords')]

#     # Repeat infinitely.
#     dataset = tf.data.TFRecordDataset(filenames).repeat()

#     # Parse records.
#     dataset = dataset.map(
#         parser)

#     # Potentially shuffle records.
#     min_queue_examples = int(NUM_PER_EPOCH * 0.4)
#     dataset = dataset.shuffle(buffer_size=min_queue_examples + 3 * batch_size)

#     # Batch it up.
#     dataset = dataset.batch(batch_size)
#     iterator = dataset.make_one_shot_iterator()
#     image_batch, label_batch = iterator.get_next()

#     return image_batch, label_batch

# sess =tf.Session()
# sess.run(tf.global_variables_initializer())
# data,label = sess.run(make_batch(5))
# print(data)


In [4]:
# def _parse_function(example_proto):
#     features = {"data": tf.FixedLenFeature((), tf.float32),
#                 "label": tf.FixedLenFeature((), tf.float32)} 
#     parsed_features = tf.parse_single_example(example_proto, features) 
# #     data = tf.decode_raw(parsed_features['data'], tf.float32)
#     return parsed_features["data"], parsed_features["label"]
# def load_tfrecords(srcfile): 
#     sess = tf.Session()
#     dataset = tf.data.TFRecordDataset(srcfile) # load tfrecord file dataset =
#     dataset.map(_parse_function) # parse data into tensor dataset = 
#     dataset.repeat(30) # repeat for 2 epoches dataset =
#     dataset.batch(5) # set batch_size = 5 iterator = 
#     iterator = dataset.make_one_shot_iterator()
#     next_data = iterator.get_next()
#     while True:
#         try: 
#             data= sess.run(next_data) 
#             print (data.data)
# #             print (label) 
#         except tf.errors.OutOfRangeError: 
#             break

            
            
# load_tfrecords(srcfile="npz_data/violence/train.tfrecords")

In [11]:
def _parse_function(example_proto):
    features = {"data": tf.FixedLenFeature((), tf.float32),
                "label": tf.FixedLenFeature((), tf.float32)} 
    parsed_features = tf.parse_single_example(example_proto, features) 

#     data = tf.decode_raw(parsed_features['data'], tf.float32)
    return parsed_features["data"], parsed_features["label"]
def load_tfrecords(srcfile): 
    sess = tf.Session()
    dataset = tf.data.TFRecordDataset(srcfile) # load tfrecord file dataset =
    dataset.map(_parse_function) # parse data into tensor dataset = 
    dataset.repeat(30) # repeat for 2 epoches dataset =
    dataset.batch(5) # set batch_size = 5 iterator = 
    iterator = dataset.make_initializable_iterator()

#     iterator = dataset.make_one_shot_iterator()
    next_element = iterator.get_next()
    return next_element,iterator



with tf.Session() as sess:
    nextData, myIter = load_tfrecords(srcfile="npz_data/violence/train.tfrecords")

    sess.run(myIter.initializer)


    try:

        mydata = sess.run(nextData.numpy())
        print(mydata.shape)
#             print(sess.run((next_data,next_label)))

    except:
    # Raised when we reach the end of the file.
        pass

<TFRecordDatasetV1 shapes: (), types: tf.string>


In [ ]:

# with tf.Session() as sess:
#     try:
#         while True:
#             print("reading")
#             data_record = sess.run(iterator.get_next())
#             print(data_record)
#     except:
#         pass

In [ ]:
def load_data(data_path):
    data=np.empty([1,frame_number,img_size[0],img_size[1],3])
    label=np.empty([1,num_class])
    file_list=glob.glob(data_path)
    data_list=[d for d in file_list if "data" in d.split("/")[-1]]
    label_list=[l for l in file_list if "label" in l.split("/")[-1]]
#     for index in range(len(data_list)):
#         d=np.load(data_list[index])["arr_0"]
#         l=np.load(label_list[index])["arr_0"]
#         data=np.append(data,d,axis=0)
#         data=np.append(label,l,axis=0)
#     data=data[1:]
#     label=label[1:]
    for index in range(len(data_list)):
        data=np.load(data_list[index])["arr_0"]
        label=np.load(label_list[index])["arr_0"]
    return data,label

data,label=load_data(data_path)

print(data.shape)

#try to upsize the data
sample=np.empty([len(data),frame_number,img_size[0],img_size[1],3])

print(sample.shape)

for frameCubeNum in range(len(data)):
    frameCube = data[frameCubeNum,...]
    frame_stack=np.empty([frame_number,img_size[0],img_size[1],3])
#     print("size of frame cube {}".format(frame_stack.shape))

    for imgcubeNum in range(len(frameCube)):
        imgcube = frameCube[imgcubeNum,...]
        img_stack=np.empty([img_size[0],img_size[1],3])
#         print("size of imgcube cube {}".format(img_stack.shape))

        for imglayerNum in range(3):
            imglayer = imgcube[...,imglayerNum]
            img_sm = cv2.resize(imglayer, (img_size[0],img_size[1]), interpolation=cv2.INTER_CUBIC)
#             print("size of imglayer {}".format(imglayer.shape))
            img_stack[...,imglayerNum] = img_sm
        
        frame_stack[imgcubeNum,...]=img_stack
#         frame_stack=np.append(frame_stack,img_stack,axis=0)
#         print(frameCube.shape)
        
    sample[frameCubeNum,...]=frame_stack
    
    
    
data = sample

print(data.shape)
print(label.shape)

In [ ]:
# #load extra test data
# def load_test_data(data_path):
#     data=np.empty([1,frame_number,img_size[0],img_size[1],3])
#     label=np.empty([1,num_class])
#     file_list=glob.glob(data_path)
#     data_list=[d for d in file_list if "data" in d.split("/")[-1]]
#     label_list=[l for l in file_list if "label" in l.split("/")[-1]]
# #     for index in range(len(data_list)):
# #         d=np.load(data_list[index])["arr_0"]
# #         l=np.load(label_list[index])["arr_0"]
# #         data=np.append(data,d,axis=0)
# #         data=np.append(label,l,axis=0)
# #     data=data[1:]
# #     label=label[1:]
#     for index in range(len(data_list)):
#         data=np.load(data_list[index])["arr_0"]
#         label=np.load(label_list[index])["arr_0"]
#     return data,label

# test2data,test2label=load_test_data(data_path2)

# print(test2data.shape)

# #try to upsize the data
# sample2=np.empty([len(test2data),frame_number,img_size[0],img_size[1],3])

# print(sample2.shape)

# for frameCubeNum in range(len(test2data)):
#     frameCube = test2data[frameCubeNum,...]
#     frame_stack=np.empty([frame_number,img_size[0],img_size[1],3])
# #     print("size of frame cube {}".format(frame_stack.shape))

#     for imgcubeNum in range(len(frameCube)):
#         imgcube = frameCube[imgcubeNum,...]
#         img_stack=np.empty([img_size[0],img_size[1],3])
# #         print("size of imgcube cube {}".format(img_stack.shape))

#         for imglayerNum in range(3):
#             imglayer = imgcube[...,imglayerNum]
#             img_sm = cv2.resize(imglayer, (img_size[0],img_size[1]), interpolation=cv2.INTER_CUBIC)
# #             print("size of imglayer {}".format(imglayer.shape))
#             img_stack[...,imglayerNum] = img_sm
        
#         frame_stack[imgcubeNum,...]=img_stack
# #         frame_stack=np.append(frame_stack,img_stack,axis=0)
# #         print(frameCube.shape)
        
#     sample2[frameCubeNum,...]=frame_stack
    
    
    
# test2data = sample2

# print(test2data.shape)
# print(test2label.shape)

In [ ]:
# train_x = data[:int(label.shape[0]*0.8),...]
# train_y = label[:int(label.shape[0]*0.8),...]
# valid_x = data[int(label.shape[0]*0.8):int(label.shape[0]*0.9),...]
# valid_y = label[int(label.shape[0]*0.8):int(label.shape[0]*0.9),...]
# test_x = data[int(label.shape[0]*0.9):,...]
# test_y = label[int(label.shape[0]*0.9):,...]

train_x, valid_x, train_y,valid_y =  train_test_split(data, label, test_size=0.3,random_state=42)
test_x, valid_x, test_y,valid_y =  train_test_split(valid_x, valid_y, test_size=0.5, random_state=42)

del data,label

In [ ]:
print(test_y.shape)


In [ ]:
import tensorflow_hub as hub


In [ ]:
# learning_rate = 1e-6
X = tf.placeholder(tf.float32,[None,frame_number,224,224,3])
labels = tf.placeholder(tf.float32,[None,num_class])


module = hub.Module("https://tfhub.dev/deepmind/i3d-kinetics-600/1",trainable=False)
outputs = module(X)

dense1 = tf.layers.dense(inputs=outputs, units=128)
dropped1 = tf.nn.dropout(dense1,keep_prob=0.5)
dense2 = tf.layers.dense(inputs=dropped1, units=128)
dropped2 = tf.nn.dropout(dense2,keep_prob=0.5)
# dense2 = tf.layers.dense(inputs=dense1, units=128)
# dense3 = tf.layers.dense(inputs=dense2, units=128)

logits = tf.layers.dense(inputs=dropped2, units=num_class)

prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=labels))
optimizer = tf.train.AdamOptimizer()
train_op = optimizer.minimize(loss_op)
        
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

saver = tf.train.Saver()

  

In [ ]:
batch_size = 2
num_steps = int(train_x.shape[0]/batch_size)
display_step = 2 #was 2 when achieved test2 accuracy 0.667

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for epoch in range(10):   #was 10
        print("epoch {}".format(epoch))

        for step in range(0, num_steps):
            batch_x, batch_y = train_x[step:(step+1)*batch_size,...],train_y[step:(step+1)*batch_size,...]

                # Run optimization op (backprop)
            sess.run(train_op, feed_dict={X: batch_x, labels: batch_y})
            if step % display_step == 0 or step == 1:
                    # Calculate batch loss and accuracy
                loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                         labels: batch_y})
                print("Step " + str(step) + ", Minibatch Loss= " + \
                          "{:.4f}".format(loss) + ", Training Accuracy= " + \
                          "{:.3f}".format(acc))


        print("Optimization Finished fpr epoch {}\n".format(epoch))
        valaccuracy =sess.run(accuracy, feed_dict={X: valid_x,
                                              labels:valid_y});
        print("validation Accuracy: {}".format(valaccuracy))
            
        f = open("result.txt","a+")
        f.write("validation accuracy for epoch {} is {}\n".format(epoch,valaccuracy))
        f.close()
    save_path = saver.save(sess, "/tmp/model.ckpt")

    del train_x,train_y,valid_x,valid_y
    testaccuracy = sess.run(accuracy, feed_dict={X: test_x,
                                              labels:test_y})
    print("Testing accuracy is {} \n".format(testaccuracy))
    f = open("result.txt","a+")
    f.write("testing accuracy is {}\n".format(testaccuracy))
    f.close()
    
    del test_x,test_y
#     print(test2label)
#     testaccuracy2 = sess.run(accuracy, feed_dict={X: test2data,
#                                               labels:test2label})
#     print("Testing2 accuracy is {} \n".format(testaccuracy2))
#     f = open("result.txt","a+")
#     f.write("testing2 accuracy is {}\n".format(testaccuracy2))
#     f.close()

 







In [ ]:
# tf.reset_default_graph()

# saver = tf.train.Saver()

# with tf.Session() as sess:
#     saver = tf.train.import_meta_graph('/tmp/model.ckpt.meta')
#     saver.restore(sess, "/tmp/model.ckpt")    
#     print("Model restored.")



In [ ]:
dataset = tf.data.Dataset.range(5)
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()

# Typically `result` will be the output of a model, or an optimizer's
# training operation.
result = tf.add(next_element, next_element)
sess = tf.Session()
sess.run(iterator.initializer)
print(sess.run(result))  # ==> "0"
print(sess.run(result))  # ==> "2"
print(sess.run(result))  # ==> "4"
print(sess.run(result))  # ==> "6"
print(sess.run(result))  # ==> "8"
try:
    sess.run(result)
except tf.errors.OutOfRangeError:
    print("End of dataset")  # ==> "End of dataset"
